In [1]:
import os
os.environ["PYSPARK_PYTHON"]="python3.7"
os.environ["PYSPARK_DRIVER_PYTHON"]="python3.7"

In [2]:
!pip install textblob
!pip install pandas
!pip install numpy

     |████████████████████████████████| 636 kB 789 kB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 258 kB/s eta 0:00:01
     |████████████████████████████████| 301 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 719 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 1.4 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434676 sha256=4694fd6661790cd1a19c15169403e756a335af7745836864cc993a18b0aeb79a
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
     |████████████████████████████████| 9.5 MB 2.9 MB/s eta 0:00:01     |████████████████████████████▊   | 8.6 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 1.8 MB/s eta 0:00:01    |███████████▌                    | 5.2 MB 3.2 MB/s eta 0:00:03     |█████████████████████           | 9.5 MB 4

In [3]:
from functools import reduce
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
            .master("spark://spark-master:7077") \
            .appName("DataSentiment") \
            .getOrCreate()

In [4]:
schema = StructType([
    StructField('comment', StringType(), False),
    StructField('sentiment', IntegerType(), False),
])

In [5]:
files = ['{:0>2}'.format(str(i)) for i in list(range(1, 5))]

In [6]:
files

['01', '02', '03', '04']

In [7]:
for file in files:
    df = spark.read.format("json").load("hdfs://namenode/user/root/input/data{name}.json".format(name=file), multiLine = "true")
    df = df.na.drop().dropDuplicates()
    df = df.filter(size(df['comments']) >= 30)
    
    comments = df.select('comments').collect()
    data = []
    from textblob import TextBlob

    for comment in comments:
        for item in comment[0]:
            try:
                sentiment = TextBlob(item).polarity
            
                if sentiment > 0:
                    data.append((item, 1))
                elif sentiment <= 0:
                    data.append((item, -1))
            except:
                # item chua cac ki tu dac biet => None
                pass
            
    df = spark.createDataFrame(data, schema)
    
    if not os.path.isfile('data_sentiment.csv'):
        df.toPandas().to_csv('data_sentiment.csv', index=False)
    else:
        df.toPandas().to_csv('data_sentiment.csv', mode='a', header=False, index=False)